# Import Modules

In [1]:
from os import listdir
from os.path import isfile, join
import os.path as path
from tempfile import mkdtemp

import netCDF4 as nc
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import rcParams
import import_ipynb
import re
import glob


import Module_Diagnostics as MD


# import Shock_understanding_preset as Sup

importing Jupyter notebook from Module_Diagnostics.ipynb
testing compute_RMS(...):
This should be close to 1: 1.0106507108855913
This should be close to 0: 0.0


# Choose run and time

In [2]:
#if working on lx-moissard
Cluster = 'Occ/'
run_name = '20_08_18_new_big_one_0'
filepath = '/data/Lathys/Visualisation/' + Cluster + run_name + '/ncfiles/p3_files/'
filepath_nc = '/data/Lathys/Visualisation/' + Cluster + run_name + '/ncfiles/'

time = 230 #TIME
time = '%05d' % time    # Change the time to string format, needed by functions


try:
    date = re.search('Magw_(.+?)_t', glob.glob(filepath_nc+'Magw*_t'+time+'.nc')[0]).group(1)
except (IndexError, AttributeError): 
    sys.exit(f"""time_dump {time} does not appear to have data.
             Let us go to the next time_dump""")

print(f'date of the simulation (DD_MM_YY): {date}')
print(f'time dump (in 1/omega_ci): {time}')

date of the simulation (DD_MM_YY): 18_08_20
time dump (in 1/omega_ci): 00230


# Download data

In [3]:
## Load B and X, Y, Z

# There might be some MemoryError. In that case, follow the guide:
# https://stackoverflow.com/questions/60563936/memoryerror-unable-to-allocate-3-78-gib-for-an-array-with-shape-802-842-1502
Magw = MD.import_data_3D(filepath_nc, date, time, 'Magw')

cwp = Magw['c_omegapi'][0]
gstep = Magw['gstep']
r_planet = Magw['r_planet'][0]

#Thomas Huret recommands shifting the axes so that the planet is indeed in (0,0,0)
#It has got something to do with lists starting at 1 instead of 0 in Fortran...?
X = np.array(np.around(Magw['x']))#+gstep[0]  
Y = np.array(np.around(Magw['y']))#+gstep[1]
Z = np.array(np.around(Magw['z']))#-gstep[2]

nx,  ny,  nz  = len(X), len(Y), len(Z)
# Location of the planet is defined in the .ncfiles as (x,y,z) = (0,0,0)
# Location of the planet is defined in the .ncfiles as (x,y,z) = (0,0,0)
nx0, ny0, nz0 = ( int(np.where(abs(X)==min(abs(X)))[0]),
                  int(np.where(abs(Y)==min(abs(Y)))[0]), 
                  int(np.where(abs(Z)==min(abs(Z)))[0])  )
            
# Use memmap to alleviate RAM
# This stores big arrays on the disk, but in a way that still allows for most
# operations available on an np.array
print("storing Magnetic field in a memmap")
file_Bx = path.join(mkdtemp(), 'Bx.dat')            
Bx = np.memmap(file_Bx, dtype='float32', mode='w+', shape=(nx,ny,nz))
Bx[:] = Magw['Bx']
file_By = path.join(mkdtemp(), 'By.dat')            
By = np.memmap(file_By, dtype='float32', mode='w+', shape=(nx,ny,nz))
By[:] = Magw['By']
file_Bz = path.join(mkdtemp(), 'Bz.dat')            
Bz = np.memmap(file_Bz, dtype='float32', mode='w+', shape=(nx,ny,nz))
Bz[:] = Magw['Bz']
print("deleting Magw to alleviate RAM")
del Magw
B = [Bx, By, Bz]

Importing Magw 3D from /data/Lathys/Visualisation/Occ/20_08_18_new_big_one_0/ncfiles/
Reading Bx...
Reading By...
Reading Bz...
Close file and return...
storing Magnetic field in a memmap
deleting Magw to alleviate RAM


In [4]:
posx = []
posy = []
posz = []

velx = []
vely = []
velz = []

i = 0

for p3_file in listdir(filepath):
    
    if not(".nc" in p3_file):
        continue
    
    i+=1
        
    print("Reading file", i, "/", len(listdir(filepath)))
    print("Reading ", p3_file, " ...")

    p3_data = nc.Dataset(filepath + p3_file,'r')
    
    posx = posx + list(np.array(p3_data.variables['particule_x']))
    posy = posy + list(np.array(p3_data.variables['particule_y']))
    posz = posz + list(np.array(p3_data.variables['particule_z']))

    velx = velx + list(-np.array(p3_data.variables['particule_vx']))
    vely = vely + list(-np.array(p3_data.variables['particule_vy']))
    velz = velz + list(np.array(p3_data.variables['particule_vz']))


Reading file 1 / 20
Reading  p3_1770_18_08_20_t00230.nc  ...
Reading file 2 / 20
Reading  p3_2849_18_08_20_t00230.nc  ...
Reading file 3 / 20
Reading  p3_1829_18_08_20_t00230.nc  ...
Reading file 4 / 20
Reading  p3_2669_18_08_20_t00230.nc  ...
Reading file 5 / 20
Reading  p3_1830_18_08_20_t00230.nc  ...
Reading file 6 / 20
Reading  p3_2190_18_08_20_t00230.nc  ...
Reading file 7 / 20
Reading  p3_2670_18_08_20_t00230.nc  ...
Reading file 8 / 20
Reading  p3_2250_18_08_20_t00230.nc  ...
Reading file 9 / 20
Reading  p3_2789_18_08_20_t00230.nc  ...
Reading file 10 / 20
Reading  p3_2790_18_08_20_t00230.nc  ...
Reading file 11 / 20
Reading  p3_2309_18_08_20_t00230.nc  ...
Reading file 12 / 20
Reading  p3_2909_18_08_20_t00230.nc  ...
Reading file 13 / 20
Reading  p3_2850_18_08_20_t00230.nc  ...
Reading file 14 / 20
Reading  p3_2189_18_08_20_t00230.nc  ...
Reading file 15 / 20
Reading  p3_1769_18_08_20_t00230.nc  ...
Reading file 16 / 20
Reading  p3_2729_18_08_20_t00230.nc  ...
Reading file 17 /

# Cubes, positions & velocities

In [5]:
# /!\ ugly hard coding

nx = 1500
ny = 720
nz = 660
dx = 1
dy = 1
dz = 1
px = (nx - 100)*dx
py = ny/2 * dy
pz = nz/2 * dz

def pos_gse(pos_simu, planet, sgn):
    return sgn*(np.array(pos_simu) - planet*np.ones(len(pos_simu)))

# Move from x_simu to x_gse
rposx = pos_gse(posx, px, -1)
rposy = pos_gse(posy, py, -1)
rposz = pos_gse(posz, pz,  1)

In [6]:
def cubes(position_shock):
    
    '''Defines two cubes: one on the left and one on the right side of
       a point.'''
    
    distance = 7
    size_cubes = 6 # Goals and poles. This will be a 5 by 5 cube.
    
    x_shock = position_shock[0]
    y_shock = position_shock[1]
    z_shock = position_shock[2]
    
    center_cube = x_shock, y_shock, z_shock

    cube_x_max = x_shock + size_cubes/2
    cube_x_min = x_shock - size_cubes/2
    cube_y_max = y_shock + size_cubes/2
    cube_y_min = y_shock - size_cubes/2
    cube_z_max = z_shock + size_cubes/2
    cube_z_min = z_shock - size_cubes/2

    ix_min_left  = int(np.where(cube_x_min + distance <=X)[0][0])    
    ix_max_left  = int(np.where(cube_x_max + distance <=X)[0][0])
    ix_min_right = int(np.where(cube_x_min - distance <=X)[0][0])    
    ix_max_right = int(np.where(cube_x_max - distance <=X)[0][0])
    iy_min = int(np.where(cube_y_min<=Y)[0][0])
    iy_max = int(np.where(cube_y_max<=Y)[0][0])
    iz_min = int(np.where(cube_z_min<=Z)[0][0])
    iz_max = int(np.where(cube_z_max<=Z)[0][0])
    
    cubes = {'left':
              { 'coord': (cube_x_min+distance, cube_x_max+distance, cube_y_min, cube_y_max, cube_z_min, cube_z_max),
                'index': (ix_min_left , ix_max_left , iy_min, iy_max, iz_min, iz_max) },
             
             'right': 
              { 'coord': (cube_x_min-distance, cube_x_max-distance, cube_y_min, cube_y_max, cube_z_min, cube_z_max),
                'index': (ix_min_right, ix_max_right, iy_min, iy_max, iz_min, iz_max) }
             }
    
    return cubes

In [7]:
def incube(point, cube):
    x, y, z = point
    xmin, xmax, ymin, ymax, zmin, zmax = cube
    return (     (xmin < x < xmax) 
             and (ymin < y < ymax)
             and (zmin < z < zmax) )

In [8]:
#for '20_08_18_new_big_one_0', at t=230
#cube = (xmin, xmax, ymin, ymax, zmin, zmax)
position_shock_sw = (-65, 180, 0)
position_shock_bs = (-63, 135, 0)
position_shock_mp = (-50,  76, 0) #current structure
#position_shock_mp = (-55,  90, 0) #clean in the msh

cubes_sw = cubes(position_shock_sw)['right']['coord']
cubes_bs = cubes(position_shock_bs)['right']['coord']
cubes_mp = cubes(position_shock_mp)['right']['coord']

In [9]:
print("Constructing list of particles in the solar wind")

x_sw = [rposx[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_sw)]
y_sw = [rposy[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_sw)]
z_sw = [rposz[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_sw)]

vA = 89.371 #Ugly hard code
vx_sw = [velx[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_sw)]
vy_sw = [vely[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_sw)]
vz_sw = [velz[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_sw)]

print("Constructing list of particles in the magnetosheath, near the bow shock")

x_bs = [rposx[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_bs)]
y_bs = [rposy[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_bs)]
z_bs = [rposz[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_bs)]

vA = 89.371 #Ugly hard code
vx_bs = [velx[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_bs)]
vy_bs = [vely[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_bs)]
vz_bs = [velz[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_bs)]

print("Constructing list of particles in the magnetosheath, near the magnetopause")

x_mp = [rposx[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_mp)]
y_mp = [rposy[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_mp)]
z_mp = [rposz[i] for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_mp)]

vA = 89.371 #Ugly hard code
vx_mp = [velx[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_mp)]
vy_mp = [vely[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_mp)]
vz_mp = [velz[i]*vA for i in range(0, len(rposx))
      if incube((rposx[i], rposy[i], rposz[i]), cubes_mp)]

Constructing list of particles in the solar wind
Constructing list of particles in the magnetosheath, near the bow shock
Constructing list of particles in the magnetosheath, near the magnetopause


In [10]:
def avg(data):
    return np.mean(data)

def rms(data):
    return np.mean(np.sqrt( (np.array(data) - np.array(avg(data)))**2))

In [11]:
# Simple geometry assumption: shock normal is along x. This is not completely true.
# Should we do something about it? It's tough because we don't have that many grid cells to play with.


def function_data_in_cube(cube_indexes, data1, data2=None, function1=MD.identity, function_both=None):
    
    '''
    Returns the mean value of data in the different cubes.
    If data is a vector (len(data)==3), then the function returns the mean value the norm(data[cubes])
    '''
                        
    ixmin, ixmax, iymin, iymax, izmin, izmax = cube_indexes
        
    slices = (slice(ixmin, ixmax), slice(iymin, iymax), slice(izmin, izmax))
        
    if (not(data2)):                       
        result = function1( data1, slices )
            
    else:
        result = function_both( function1( data1, slices ), [data2[0][slices],
                                                             data2[1][slices],
                                                             data2[2][slices]] )
    data_in_cube = []        
    if (len(result) == 3):
        data_in_cube.append( (np.nanmean(result[0]), np.nanmean(result[1]), np.nanmean(result[2])) )
    else:
        data_in_cube.append( np.nanmean(result) )

        

    return data_in_cube

In [12]:
Bx_sw = function_data_in_cube(cubes(position_shock_sw)['right']['index'], Bx)[0]
By_sw = function_data_in_cube(cubes(position_shock_sw)['right']['index'], By)[0]
Bz_sw = function_data_in_cube(cubes(position_shock_sw)['right']['index'], Bz)[0]

Bx_bs = function_data_in_cube(cubes(position_shock_bs)['right']['index'], Bx)[0]
By_bs = function_data_in_cube(cubes(position_shock_bs)['right']['index'], By)[0]
Bz_bs = function_data_in_cube(cubes(position_shock_bs)['right']['index'], Bz)[0]

Bx_mp = function_data_in_cube(cubes(position_shock_mp)['right']['index'], Bx)[0]
By_mp = function_data_in_cube(cubes(position_shock_mp)['right']['index'], By)[0]
Bz_mp = function_data_in_cube(cubes(position_shock_mp)['right']['index'], Bz)[0]


# Energy & particle count

In [ ]:
eV = MD.qe #J

E_sw = (1./2) * ((np.array(vx_sw)*1e3)**2 + (np.array(vy_sw)*1e3)**2 + (np.array(vz_sw)*1e3)**2) * MD.mp / eV
E_bs = (1./2) * ((np.array(vx_bs)*1e3)**2 + (np.array(vy_bs)*1e3)**2 + (np.array(vz_bs)*1e3)**2) * MD.mp / eV
E_mp = (1./2) * ((np.array(vx_mp)*1e3)**2 + (np.array(vy_mp)*1e3)**2 + (np.array(vz_mp)*1e3)**2) * MD.mp / eV

E_sw = 1e-3*E_sw #keV
E_bs = 1e-3*E_bs #keV
E_mp = 1e-3*E_mp #keV

# Plots

In [13]:
def plot_zero_lines():
    ax = plt.gca()
    ax.axhline(y=0, color="gray", linestyle='--', linewidth=0.5)
    ax.axvline(x=0, color="gray", linestyle='--', linewidth=0.5)

In [21]:
rcParams["figure.figsize"] = [3, 2]
%matplotlib

# Keel all axes the same to facilitate visual comparison
max_x = max(avg(vx_mp)+8*rms(vx_mp), avg(vx_mp)-8*rms(vx_mp))
max_y = max(avg(vy_mp)-8*rms(vy_mp), avg(vy_mp)+8*rms(vy_mp))

xlim = [max_y, -max_y]
ylim = [-max_y, max_y]

plt.close('all')
plt.figure()

ax1 = plt.subplot(311)
plot_zero_lines()
plt.scatter(vx_sw, vy_sw, s = 0.05, color='black')
# plt.title("Velocity distribution in the solar wind", fontsize=17, weight="bold", color="red")
plt.xlabel(r"$V_x$ (km/s)", fontsize=15)
plt.ylabel(r"$V_y$ (km/s)", fontsize=15)
ax1.set_aspect('equal')
ax1.yaxis.tick_right()
ax1.yaxis.set_label_position("right")
# plt.xlim([avg(vx_sw)+8*rms(vx_sw), avg(vx_sw)-8*rms(vx_sw)])
# plt.ylim([avg(vy_sw)-8*rms(vy_sw), avg(vy_sw)+8*rms(vy_sw)])


ax2 = plt.subplot(312, sharex=ax1, sharey=ax1)
plot_zero_lines()
plt.scatter(vx_bs, vy_bs, s = 0.05, color='black')
# plt.title("Velocity distribution in the magnetosheath, near the bow shock", fontsize=17, weight="bold", color="red")
plt.xlabel(r"$V_x$ (km/s)", fontsize=15)
plt.ylabel(r"$V_y$ (km/s)", fontsize=15)
ax2.set_aspect('equal')
ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")
# plt.xlim([avg(vx_bs)+8*rms(vx_bs), avg(vx_bs)-8*rms(vx_bs)])
# plt.ylim([avg(vy_bs)-8*rms(vy_bs), avg(vy_bs)+8*rms(vy_bs)])

ax3 = plt.subplot(313, sharex=ax1, sharey=ax1)
plot_zero_lines()
plt.scatter(vx_mp, vy_mp, s = 0.15, color='black')
# plt.title("Velocity distribution in the magnetosheath, near the magnetopause", fontsize=17, weight="bold", color="red")
plt.xlabel(r"$V_x$ (km/s)", fontsize=15)
plt.ylabel(r"$V_y$ (km/s)", fontsize=15)
ax3.set_aspect('equal')
ax3.yaxis.tick_right()
ax3.yaxis.set_label_position("right")
# ax.quiver(-1643, 73, Bx_mp, By_mp, units='xy', scale_units='width', color='red')
plt.xlim(xlim)
plt.ylim(ylim)

# plt.tight_layout()
plt.show()

Using matplotlib backend: TkAgg


In [22]:
rcParams["figure.figsize"] = [3, 2]

# Keel all axes the same to facilitate visual comparison
xlim = [max_y, -max_y]
ylim = [-max_y, max_y]

plt.close('all')
plt.figure()

ax1 = plt.subplot(311)
plot_zero_lines()
plt.scatter(vx_sw, vz_sw, s = 0.05, color='black')
# plt.title("Velocity distribution in the solar wind", fontsize=17, weight="bold", color="red")
plt.xlabel(r"$V_x$ (km/s)", fontsize=15)
plt.ylabel(r"$V_z$ (km/s)", fontsize=15)
ax1.set_aspect('equal')
ax1.yaxis.tick_right()
ax1.yaxis.set_label_position("right")
# plt.xlim([avg(vx_sw)+8*rms(vx_sw), avg(vx_sw)-8*rms(vx_sw)])
# plt.ylim([avg(vy_sw)-8*rms(vy_sw), avg(vy_sw)+8*rms(vy_sw)])

ax2 = plt.subplot(312, sharex=ax1, sharey=ax1)
plot_zero_lines()
plt.scatter(vx_bs, vz_bs, s = 0.05, color='black')
# plt.title("Velocity distribution in the magnetosheath, near the bow shock", fontsize=17, weight="bold", color="red")
plt.xlabel(r"$V_x$ (km/s)", fontsize=15)
plt.ylabel(r"$V_z$ (km/s)", fontsize=15)
ax2.set_aspect('equal')
ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")
# plt.xlim([avg(vx_bs)+8*rms(vx_bs), avg(vx_bs)-8*rms(vx_bs)])
# plt.ylim([avg(vy_bs)-8*rms(vy_bs), avg(vy_bs)+8*rms(vy_bs)])

ax3 = plt.subplot(313, sharex=ax1, sharey=ax1)
plot_zero_lines()
plt.scatter(vx_mp, vz_mp, s = 0.15, color='black')
# plt.title("Velocity distribution in the magnetosheath, near the magnetopause", fontsize=17, weight="bold", color="red")
plt.xlabel(r"$V_x$ (km/s)", fontsize=15)
plt.ylabel(r"$V_z$ (km/s)", fontsize=15)
ax3.set_aspect('equal')
ax3.yaxis.tick_right()
ax3.yaxis.set_label_position("right")
# ax.quiver(-1643, 73, Bx_mp, By_mp, units='xy', scale_units='width', color='red')
plt.xlim(xlim)
plt.ylim(ylim)

# plt.tight_layout()
plt.show()

In [ ]:
rcParams["figure.figsize"] = [3, 2]

xlim = [0, np.mean(E_mp)+6*np.std(E_mp)]

plt.close('all')
fig = plt.figure()

ax1 = plt.subplot(311)
plt.hist(E_sw, bins=100, range=xlim)
plt.ylabel(r"Nb", fontsize=15)
plt.yscale("log")
ax1.get_xaxis().set_visible(False)

ax2 = plt.subplot(312, sharex=ax1)
plt.hist(E_bs, 100, range=xlim)
plt.ylabel(r"Nb", fontsize=15)
plt.yscale("log")
ax2.get_xaxis().set_visible(False)

ax3 = plt.subplot(313, sharex=ax1)
plt.hist(E_mp, 100, range=xlim)
plt.xlabel(r"E_part (keV)", fontsize=15)
plt.ylabel(r"Nb", fontsize=15)
plt.yscale("log")

plt.xlim(xlim)

fig.subplots_adjust(wspace=0)
fig.subplots_adjust(hspace=0)

# plt.tight_layout()
plt.show()



In [17]:
size_cubes = 6

for v in [vy_sw, vy_bs, vy_mp]:
    if len(v) < 5*(size_cubes-1)**3:
        print(f"Be careful interpreting the results, there are only {len(v)} particles for {5*(size_cubes-1)**3} cells")